Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## 1. Подготовка

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import lightgbm as lgb
import lightgbm
import time 
import re
import nltk

from catboost import CatBoostRegressor
from catboost import cv
from sklearn.model_selection import train_test_split 
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn. model_selection import LeaveOneOut
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from lightgbm import LGBMRegressor, LGBMClassifier, Booster
from numpy import mean
from numpy import absolute
from numpy import sqrt
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 
from pymystem3 import Mystem
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from nltk.stem import WordNetLemmatizer
from nltk.corpus import brown
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

RANDOMSTATE = 12345
STOPWORDS = set(nltk_stopwords.words('english'))

def clear_text(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

#def lemmatize(text):
#    m = Mystem()
#    lemm_list = m.lemmatize(text)
#    lemm_text = "".join(lemm_list)
#    return lemm_text

lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')

def lemmatize(text): 
    m = nltk.word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(x) for x in m])

Подключил библиотеки и определил переменные.

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')

Загрузил датасет и сохранил его в переменной data.

In [4]:
data.head(15)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


Просмотрел первые 5 значения в датасете, вижу что текст на английском языке, в тексте присутствуют разделите, знаки препинания, буквы в различном регистре.

In [5]:
data['text'] = data['text'].apply(clear_text)

Преобразовал текст с помощью функции.

In [6]:
data['text'].head(5)

0    explanation why the edits made under my userna...
1    daww he matches this background colour im seem...
2    hey man im really not trying to edit war its j...
3    more i cant make any real suggestions on impro...
4    you sir are my hero any chance you remember wh...
Name: text, dtype: object

Повторно просмотрел первые 5 значения в датасете, визуально качество текста улучшилось.

In [8]:
corpus = []
for x in data['text']: 
    corpus.append(lemmatize(x))

Лемматизировал текст.

In [9]:
print(corpus[0])

explanation why the edits made under my username hardcore metallica fan were reverted they werent vandalism just closure on some gas after i voted at new york doll fac and please dont remove the template from the talk page since im retired now


Просмотрел текст.

In [10]:
features = corpus
target = data['toxic']

Выделил признаки.

In [11]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.25, random_state = RANDOMSTATE)

Разбил выборку на обучающую и тестовую.

### Вывод

В первой части своего проекта, я загрузил данные, проанализировал их, выполнил предобработку и подготовил выборки для обучения.

## 2. Обучение

In [12]:
count_tf_idf = TfidfVectorizer(stop_words = STOPWORDS)

features_train = count_tf_idf.fit_transform(features_train)
features_test = count_tf_idf.transform(features_test)

Определил оценку TF-IDF.

### 2.1 RandomForestClassifier

In [14]:
%%time

pipeline = Pipeline([('selector', SelectKBest()), ('model', RandomForestClassifier(random_state=RANDOMSTATE, class_weight='balanced'))])
model_3 = GridSearchCV(estimator = pipeline, param_grid = {'selector__k':[3,4,5,6,7,8,9,10]}, n_jobs=-1, scoring='f1', cv=5, verbose=3)
model_3.fit(features_train, target_train)
model_3.best_estimator_.fit(features_train, target_train)

print('Значение RMSE для RandomForestClassifier: {: 0.3f}'.format(model_3.best_score_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Значение RMSE для RandomForestClassifier:  0.567
CPU times: total: 5.66 s
Wall time: 28.4 s


### 2.2 LogisticRegression

In [15]:
%%time

model_2 = LogisticRegression(random_state=RANDOMSTATE, class_weight='balanced')
param_grid = {'C': [0.01, 0.1, 1, 10], 'max_iter': [1000], 'solver':['lbfgs']}
model_2 = GridSearchCV(model_2, param_grid, cv=5, verbose=1, scoring='f1')
model_2.fit(features_train, target_train)
model_2.best_estimator_.fit(features_train, target_train)

print('Значение f1 для LogisticRegression: {: 0.3f}'.format(model_2.best_score_))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Значение f1 для LogisticRegression:  0.760
CPU times: total: 6min 18s
Wall time: 2min 8s


### Вывод

Во второй части своего проекта для тестовой и обучающей выборок я вычислил значение TF-IDF. Далее я обучил две модели RandomForestClassifier и LogisticRegression, для обучения я использовал различные гиперпараметры и учил модели на лучших параметрах.  
Модель RandomForestClassifier показала результат меньше чем LogisticRegression.  
Для проверки на тестовой выборке я возьму модель LogisticRegression.

## 3. Выводы

In [16]:
%%time
test = f1_score(target_test, model_2.predict(features_test))

print('Значение f1 для LogisticRegression на test: {: 0.3f}'.format(test))

Значение f1 для LogisticRegression на test:  0.770
CPU times: total: 46.9 ms
Wall time: 15 ms


В третье части своего проекта я выполнил тестирование лучшей модели LogisticRegression. Значение f1 метрики на тестовой выборке составило 0.770, что является достаточным значением для успешного завершения работы.